<a href="https://colab.research.google.com/github/stanfordstuckey/Classwork/blob/main/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 6: Clinical Datasets and Predictive Modeling

In this assignment you will gain experience analyzing preprocessed clinical datasets. You will practice using common time-saving tools in the `R` programming language that are ideally suited to these tasks. 

You will work with a dataset that we have prepared for you using a process similar to what you were given in the prior homework. The dataset describes patients from the [MIMIC III database](https://mimic.physionet.org/mimictables/patients/) who were put on mechanical ventilation and were stable for 12 hours. Some of these patients then experienced a sudden and sustained drop in oxygenation, while others did not. 

We have recorded a variety of features about each patient before the 12-hour mark (the index time), including counts of all prior diagnoses (aggregated with IC), all respiratory-related concepts in their notes, and indicators of events recorded in the patient charts. Indicator features are the number of times each event was recorded in the patient record, regardless of what the measured value was. For those chart events which have numeric values associated wtih them (e.g. lab tests) we found those in which a value was recorded for over 85% of the cohort and included the latest recorded value of those features. In addition, we have included demographic features (age and sex). For the small number of patients who did not have one or more of those features recorded, we used column-mean imputation to impute them. We also recorded whether or not each patient went on to experience a sudden and sustained drop in their oxygenation (the exposure). Finally, we recorded whether or not each patient eventually died during their hospitalization (the outcome). All of that data is contained in `patient_feature_matrix.csv`. Its companion file `feature_descriptions.csv` has descriptions of each of the features and their provenance. The final dataset you have access to is called `cohort.csv`, which contains the index time, exposure time (if any), in-hospital time of death (if any), and the time of censoring (when the patient was released from the hospital).

The first portion of this assignment is identical to the first portion of the inferential analyses assignment.

Please edit this document directly using either Jupyter Notebook or R markdown in R Studio and answer each of the questions below in-line. Jupyter and R markdown are useful tools for reproducible research that you will use over and over again in your later work. They are worth taking the short amount of time necessary to learn them. Turn in a single `.pdf` document showing all of your code and output for the entire assignment, with each question clearly demarcated. Submit your completed assignment through Canvas.

**Grading**: All answers will be graded on the correctness and quality of your code and analyses. Partial credit will be given based on a demonstration of conceptual understanding and how close you can come to solving the problem. At various points we will ask you to produce particular values: the correctness of these numbers will not be used for your grade - they are tools for us to get an idea about what your code is doing.

-----

## 0. Getting Ready

The first thing we need to do is load all of the important packages we will use for this assignment. Please load the packages  `caret` and `tidyverse`. There are several other packages you will need or may want to use during the course of the assignment but if you need a package other than one of these two for a particular problem it will be noted in the problem statement.

-----

In [1]:
install.packages("caret")
install.packages("tidyverse")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘future.apply’, ‘progressr’, ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘gower’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
library(caret)
library(tidyverse)

Loading required package: ggplot2

Loading required package: lattice

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ purrr::lift()   masks caret::lift()



Next, load the CSV files `patient_feature_matrix.csv`, `cohort.csv` and `feature_descriptions.csv` as data frames.

-----

## 1. (0 pts) Preprocessing

### 1.1 (0 pts) Creating Feature Matrix and Outcome Vector

Split the patient matrix up into a numerical matrix of features and a character vector of the outcome (died or survived). For the feature matrix, exclude the subject ID and the outcome variable and use `data.matrix()`. 

----

### 1.2 (0 pts) Removing Uninformative Features

Before we do any modeling, let's cut down on our feature space by removing low-variance features that probably aren't useful enough to measure association with or use in a predictive model. `caret` has a function to do that, so let's use it instead of reinventing the wheel. 

Find the relevant function in the `caret` documentation and use it to create a new patient-feature matrix with only the useful features. From now on we will use the result of this step instead of the full feature matrix. 

----

## 2. (100 pts) Predictive Analyses

In this part of the assignment we will see if we can predict which patients will die during their hospitalizations,  given only the data from before the end of their 12-hour long stable ventiliation period.

We will use the [`caret`](caret.r-forge.r-project.org) library for our predictive modeling tasks, so take a minute to acquaint yourself with it. 

*Note on packages and masking*: You will load many packages in these exercises and some of them will import functions with names that conflict with other functions. To call the function from the package that you want, you can use the `::` qualifier as in `package::function()`. For example, if both `dplyr` and `plyr` are loaded, you would use `dplyr::summarize()` to call `dplyr`'s summarize and `plyr::summarize()` to call `plyr`'s summarize.

----

### 2.1 (5 pts)  Creating training and test sets

To find out how good the predictive models we will make are, we'll need to randomly split the data into training and test sets. 

Use `caret` to make training and test sets that preserve the proportions of the outcome in each dataset. Use an 80% training / 20% testing split. 

----

### 2.2 (30 pts) Exploratory Modeling

#### 2.2.1 (10 pts) Exploratory Elastic Net

Fit an elastic net model on the training data and use it to predict on the test set. Use $\lambda=0.01$ and $\alpha=1$ (LASSO). 

What is the missclassification accuracy of the resulting model?

-----

#### 2.2.2 (5 pts) Performance Metrics

This looks like good performance, but misclassification accuracy can be misleading. It would be useful to look at the two-by-two table for the predictions vs. the true outcomes. `caret` has a function for this that also computes useful metrics to measure classifier performance. Find it in the documentation and use it to find the sensitivity and specificity of this model. What are those values for this model? What is a simple strategy or rule you could use to get a reasonable misclassification accuracy in this case without using any training data at all?

-----

#### 2.2.3 (10 pts) ROC and PR Curves

Alternatively, we can use the predicted class *probabilities* instead of the predicted *classes* and use either a precision-recall or ROC curve to assess the model. 

Write your own code and use `ggplot2` to generate both an ROC curve and a precision-recall curve for the performance of this model on the test set.

-----

#### 2.2.4 (5 pts) Calibration Plot

Another good way to assess the utility of a classifier is with a calibration plot.

Write your own code and use `ggplot2` to generate a calibration plot for the performance of this model on the test set.

-----

### 2.3 (30 pts) Cross-Validation with the Elastic Net

#### 2.3.1 (10 pts) Cross-Validating the Elastic Net in Caret

Let's see if we can find a different model that will do better by searching over different values of $\gamma$ and $\alpha$. To assess the utility of each model, we will use four-fold cross validation over the training set and calculate the AUC (note that `caret` annoyingly refers to the AUC as the ROC) on each held-out set. At the end, we will use the model parameters that give the best result and see how well the model does on the test set. We will test over a grid of $\lambda$ and $\alpha$ values. Use $\lambda \in e^{[-6.5, -6, -5.5 ... -3, -2.5, -2]}$ and $\alpha \in [0.1, 0.5, 0.9]$.

Use `caret` to do this cross-validation and produce a `caret` model object using the functions `trainControl`, `expand.grid`, and `train`. You will likely have to search google and the `caret` documentation to find out how to do this and how to set the metric to AUC. Depending on your computer, fitting these models could take a few minutes. Call `plot()` on the returned model object to examine how the AUC changes with the different parameter choices.  

Examine the returned model object to find the values of $\alpha$ and $\lambda$ that produced the best result. What were they, and what was the AUC of that model? (*hint: the model object contains two dataframes that you can inner join that will neatly give you this result*)

-----

#### 2.3.2 (5 pts) Model Performance

Use the `pROC` package to calculate the AUC of the ROC curve for this model. What is the AUC statistic of this model on the test set? Is it close to what was estimated by cross-validation?

-----

#### 2.3.3 (5 pts) Test Error Estimation After Feature Selection

What if we had used univariate analyses on the training set to find all of the features that are significantly associated with mortality and have a large enough effect size and then used only those features to fit our models? Would you expect the cross-validation AUC to be on average larger, smaller, or the same as the test set AUC? Why? If you needed to reduce the feature space in this way, what could you in the cross validation to avoid potential mis-steps?



#### 2.3.4 (5 pts) Comparing Feature Selection Methods

Is what is decribed above in 2.3.3 the same as we did earlier by removing the near-zero-variance features in the sense that we should we expect it to have a similar effect on the test vs. cross-validation AUC? Why or why not?


#### 2.3.5 (5 pts) Inspecting Coefficients

For the best model, what are the 10 most important features (in terms of the magnitude of their coefficients) and their descriptions and feature types? You will need to call the `coef` function on an internal datastructure of the `caret` object to do this. As always, `dplyr` comes in handy as well.

-----

### 2.4 (30 pts) Cross Validation with Gradient Boosted Trees

#### 2.4.1 (5 pts) Parameters for Gradient Boosted Trees

Let's see if we can do better with a nonlinear model. Gradient boosted trees are considered to be state-of-the-art, so let's give them a shot. In `caret`, boosted trees are implemented in the `gbm` method, which has several parameters. Describe each parameter and whether increasing each of them increases or decreases the bias or variance.


#### 2.4.2 (5 pts) Cross-Validating Gradient Boosted Trees in Caret

Using the same evaluation metric (ROC) and the same cross-validation setup (4-fold CV) as before, fit a gradient boosted tree model using `caret`. Set the interaction depth at 3, the minimum observations per node at 3, and the shrinkage at 0.1. Fit models ranging from 5 to 250 trees, in increments of 5 trees. Training may take some time depending on your computer. Plot the model object to see how the cross-validation AUC changes as more trees are fit, and report the best parameter set and the resulting AUC, sensitivity, and specificity. 

-----

#### 2.4.3 (5 pts) Model Performance

Plot the test set ROC curve for this model.

-----

#### 2.4.4 (5 pts) Variable Importance

Use the `varImp` function to find the top ten most important features in this model. Report the importance measures, descriptions, and feature types for these top ten features. Read up about how these importance measures are calculated. Would a variable that is only split on in the first tree be more important than a variable that is only split on in the 200th tree? Which types of feature seem to be the most useful? Why do you think that is the case?

-----

#### 2.4.5 (5 pts) Partial Dependence Plots

One of the nice things about tree ensembles is that they can automatically find and exploit interactions between features. Use the final model object and the plot command exported by the `gbm` package to plot the two-way partial dependence of the outcome on `age_in_days` and `chartvalue_198` as well as the one-way partial dependance on each of them individually. Read a bit about partial dependence plots. Is the effect of age linear? What combination of these two features is most associated with worse outcomes? Do you think the result makes sense? Why or why not?

-----

#### 2.4.6 (5 pts) Investigating the Effect of Age

What is strange about the range of the age variable in the partial dependence plot? Investigate by plotting a histogam of patient ages. What does this say about the data? Do you think this adversely affects the test set peformance of the gradient boosted tree model? What do you think the effect is on the linear model? Do you have evidence of that effect given the coefficients of the linear model?

-----

### 2.5 (5 pts) Proper Evaluation of Predictive Models

Given that we randomly split the data into training and test sets, do you think that the test set accuracy would be a good estimate, an overestimate, or an underestimate of the accuracy if we used this model to predict mortality for patients in the coming year? Justify your answer in one or two sentences. Give a suggestion for an alternative data-splitting method that could be better and why.

## Feedback (0 points)
####  How much time did you spend on this assignment?

####  How much did you learn? Choose one (type your answer after the table):
   A | B | C | D | E |
   --|---|---|---|---|
   a great deal |  a lot  |  a moderate amount | a little | none at all|

#### Did you do any of the following: go to office hours, post on canvas, e-mail TAs? If so, which?